In [ ]:
# Fijar semilla
np.random.seed(42)

# Cantidad de nodos
n_nodes = 3

# Cantidad de estados por nodo
n_states = 3

# Cantidad de estados
nx = n_nodes * n_states

# Parámetros 
beta = 1.
gamma = 0.3

# Parámetros de interacción entre nodos
lambdas = stats.uniform(0, 1).rvs(size=(n_nodes, n_nodes))

# Función de dinámica
def f(x):
    to_reshape = False
    if len(x.shape) == 1:
        x = x.reshape((nx, 1))
        to_reshape = True

    S, I, R = x[:n_nodes, :], x[n_nodes:2*n_nodes, :], x[2*n_nodes:, :]
    
    incidence = np.array([
        np.sum([lambdas[i, j]*I[j, :] for j in range(n_nodes)], axis=0) for i in range(n_nodes)
    ])

    susceptibles = S - beta*incidence*S
    infected = I + beta*incidence*S - gamma*I
    recovered = R + gamma*I

    if to_reshape:
        return np.concatenate([susceptibles, infected, recovered]).squeeze()
    else:
        return np.concatenate([susceptibles, infected, recovered])

# Función de observación
def g(x):
    return x

# Covarianzas del ruido
sigma = 0.01
Q = sigma*np.eye(nx)
R = sigma*np.eye(nx)

# Distribuciones
X_dist = stats.dirichlet(alpha=np.array([1, 1, 1, 1, 1, 1, 1, 1, 1]))
dyn_dist = stats.multivariate_normal(mean=np.zeros(nx), cov=Q)
obs_dist = stats.multivariate_normal(mean=np.zeros(nx), cov=R)

# Condición inicial 
x0 = X_dist.rvs().squeeze()

# Condición inicial prior
x0_prior = stats.multivariate_normal(mean=x0, cov=0.01*np.eye(nx))

# Número de dimensiones de aproximación
N = 2500

# Número de iteraciones
iters = 20

# Operador de Koopman
dyn = DynamicalSystem(nx, nx, f, g, X_dist, dyn_dist, obs_dist, discrete_time=True)
koopman = KoopmanOperator(kernel_function=Matern(length_scale=1e3, nu=0.5), dynamical_system=dyn)

# Trayectoria real y observaciones
x = np.zeros((nx, iters))
y = np.zeros((nx, iters))
x[:, 0] = x0

for i in range(1, iters):
    x[:, i] = f(x[:, i-1]) 
    y[:, i] = g(x[:, i]) + obs_dist.rvs()

# KKKF
sol_kkkf = kkkf.apply_koopman_kalman_filter(koopman, y.reshape((iters, nx)), x0_prior, N)

In [ ]:
# # Gráfico por cada nodo
# fig, ax = plt.subplots(n_nodes, 1, figsize=(12, 12))

# for i in range(n_nodes):
#     ax[i].plot(x[i, :], 'o', label='S (Real)')
#     ax[i].plot(sol_kkkf.x_plus.T[i, :], label='S (KKKF)')
#     ax[i].plot(x[i+n_nodes, :], 'o', label='S (Real)')
#     ax[i].plot(sol_kkkf.x_plus.T[i+n_nodes, :], label='S (KKKF)')
#     ax[i].plot(x[i+2*n_nodes, :], 'o', label='S (Real)')
#     ax[i].plot(sol_kkkf.x_plus.T[i+2*n_nodes, :], label='S (KKKF)')
#     ax[i].set_title(f'Nodo {i+1}')
#     ax[i].legend()

In [ ]:
# Fijar semilla
np.random.seed(41)

# Cantidad de nodos
n_nodes = 2

# Cantidad de estados por nodo
n_states = 3

# Cantidad de estados
nx = n_nodes * n_states + n_nodes**2
ny = n_nodes * n_states

# Parámetros 
beta = 1.
gamma = 0.3

# Parámetros de interacción entre nodos
lambdas_real = stats.uniform(0, 1).rvs(size=(n_nodes, n_nodes))

# Función de dinámica
def f(x):
    to_reshape = False
    if len(x.shape) == 1:
        x = x.reshape((nx, 1))
        to_reshape = True

    S, I, R, L = x[:n_nodes, :], x[n_nodes:2*n_nodes, :], x[2*n_nodes:3*n_nodes, :], x[3*n_nodes:, :]
    lambdas = L.reshape((n_nodes, n_nodes, x.shape[1]))
    
    incidence = np.array([
        np.sum([lambdas[i, j, :]*I[j, :] for j in range(n_nodes)], axis=0) for i in range(n_nodes)
    ])

    susceptibles = S - beta*incidence*S
    infected = I + beta*incidence*S - gamma*I
    recovered = R + gamma*I

    if to_reshape:
        return np.concatenate([susceptibles, infected, recovered, L]).squeeze()
    else:
        return np.concatenate([susceptibles, infected, recovered, L])

# Función de observación
def g(x):
    to_reshape = False
    if len(x.shape) == 1:
        x = x.reshape((nx, 1))
        to_reshape = True
    
    if to_reshape:
        return x[:3*n_nodes,:].squeeze()
    else:
        return x[:3*n_nodes,:]

# Covarianzas del ruido
sigma = 0.01
Q = sigma*np.eye(nx)
R = sigma*np.eye(ny)

# Distibución Dirichlet en las primera 3 y uniforme en las otras 2
class DirichletUniform(stats.rv_continuous):
    def __init__(self, alpha, momtype=1, a=None, b=None, xtol=1e-14, badvalue=None, name=None, longname=None, shapes=None, seed=None):
        super().__init__(momtype, a, b, xtol, badvalue, name, longname, shapes, seed)
        self.alpha = alpha
    
    def rvs(self, size=1):
        x = stats.dirichlet(self.alpha).rvs(size=size)
        y = stats.uniform(0, 1).rvs(size=(size,n_nodes**2))

        return np.vstack([x.T, y.T]).T

# Distribuciones
X_dist = DirichletUniform(alpha=np.array([1, 1, 1, 1, 1, 1]))
dyn_dist = stats.multivariate_normal(mean=np.zeros(nx), cov=Q)
obs_dist = stats.multivariate_normal(mean=np.zeros(ny), cov=R)

# Condición inicial 
x0 = X_dist.rvs().squeeze()
x0_ = np.concatenate([x0[:3*n_nodes], stats.uniform(0, 1).rvs(size=n_nodes**2)])

# Condición inicial prior
x0_prior = stats.multivariate_normal(mean=x0_, cov=10*np.eye(nx))

# Número de dimensiones de aproximación
N = 2000

# Número de iteraciones
iters = 10

# Operador de Koopman
dyn = DynamicalSystem(nx, nx, f, g, X_dist, dyn_dist, obs_dist, discrete_time=True)
koopman = KoopmanOperator(kernel_function=Matern(length_scale=1e3, nu=0.5), dynamical_system=dyn)

# Trayectoria real y observaciones
x = np.zeros((nx, iters))
y = np.zeros((ny, iters))
x[:, 0] = x0

for i in range(1, iters):
    x[:, i] = f(x[:, i-1]) 
    y[:, i] = g(x[:, i]) + obs_dist.rvs()

# KKKF
# sol_kkkf = kkkf.apply_koopman_kalman_filter(koopman, y.T, x0_prior, N)

# it = 100
# params = np.zeros((it, n_nodes**2))
# sigmas = np.zeros((it, n_nodes**2))

# params[0] = sol_kkkf.x_plus[-1, 3*n_nodes:]
# sigmas[0] = np.diag(sol_kkkf.Px_plus[-1])[3*n_nodes:]

# x_ = x0_
# P = sol_kkkf.Px_plus[-1]

# params[0] = x0_[3*n_nodes:]

# for i in range(1, it):
#     x0_prior = stats.multivariate_normal(mean=x_, cov=Q)
#     dyn_dist = stats.multivariate_normal(mean=np.zeros(nx), cov=Q)
#     dyn = DynamicalSystem(nx, ny, f, g, X_dist, dyn_dist, obs_dist, discrete_time=True)
#     koopman = KoopmanOperator(kernel_function=Matern(length_scale=1e3, nu=0.5), dynamical_system=dyn)

#     sol_kkkf = kkkf.apply_koopman_kalman_filter(koopman, y.T, x0_prior, N)
#     P = sol_kkkf.Px_plus[-1]

#     params[i] = sol_kkkf.x_plus[-1, 3*n_nodes:]
#     x_ = np.concatenate([x0[:3*n_nodes], sol_kkkf.x_plus[-1,3*n_nodes:]])

In [ ]:
p = 1e-3
subr = 0.8

# Función que implementa Reed-Frost
def reed_frost(x, w):
    """
    Implementación del modelo Reed-Frost.
    
    Args:
    x: np.array
        Estado del sistema.
    w: float
        Realización de una variable aleatoria uniforme en [0, 1].

    Returns:
    x_next: np.array
        Próximo estado del sistema.
    """

    # Número de individuos
    S, I = x[0], x[1]
    N = S + I

    # Probabilidad de infección
    p_inf = 1 - (1 - p)**(I/N)

    # Número de infectados
    new_I = stats.binom(n=S, p=p_inf).ppf(w)

    # Próximo estado
    x_next = np.array([S - new_I, I + new_I])

    return x_next

# Función de observación, que también será una binomial
def obs(x, v):
    """
    Función de observación para el modelo Reed-Frost.
    
    Args:
    x: np.array
        Estado del sistema.
    v: float
        Realización de una variable aleatoria uniforme en [0, 1].

    Returns:
    y: np.array
        Observación del sistema.
    """

    # Estados
    S, I = x[0], x[1]

    # Observación
    y = stats.binom(n=I, p=subr).ppf(v)

    return np.array([y])

class IntDirichlet:
    def __init__(self, alpha, N):
        self.alpha = alpha
        self.N = N

    def rvs(self, size=1):
        samples = stats.dirichlet.rvs(self.alpha, size=size) 
        for i in range(len(self.alpha)-1):
            samples[:,i] = np.round(self.N*samples[:,i])

        samples[:,-1] = self.N - np.sum(samples[:,:-1], axis=1)
        return samples
    
X_dist = IntDirichlet(alpha=np.array([1, 1]), N=100)
dyn_dist = stats.uniform(0, 1)
obs_dist = stats.uniform(0, 1)

from KKKF.DynamicalSystems import create_additive_system

# Crear sistema dinámico aditivo
dyn = create_additive_system(2, 1, reed_frost, obs, X_dist, dyn_dist, obs_dist, N_samples=30)

# Operador de Koopman
koopman = KoopmanOperator(kernel_function=Matern(length_scale=0.1, nu=0.5), dynamical_system=dyn)

# Trayectoria real y observaciones
x = np.zeros((2, iters))
y = np.zeros((1, iters))
x[:, 0] = np.array([20, 80])
for i in range(1, iters):
    x[:, i] = reed_frost(x[:, i-1], np.random.uniform(0, 1)) 
    y[:, i] = obs(x[:, i], np.random.uniform(0, 1))

# Inicialización de las covarianzas iniciales
x0_prior = IntDirichlet(alpha=np.array([0.9, 0.1]), N=100)
P0 = 0.1*np.eye(2)

N = 500

# KKKF
#sol_kkkf = kkkf.apply_koopman_kalman_filter(koopman, y.T, x0_prior, N)